In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [2]:
# 1. Load dataset
ratings = pd.read_csv("dataset/ratings.csv")
movies = pd.read_csv("dataset/movies.csv")

In [3]:
# Ánh xạ userId và movieId về chỉ số liên tục
user_mapping = {id: idx for idx, id in enumerate(ratings["userId"].unique())}
movie_mapping = {id: idx for idx, id in enumerate(ratings["movieId"].unique())}

ratings["userId"] = ratings["userId"].map(user_mapping)
ratings["movieId"] = ratings["movieId"].map(movie_mapping)


In [4]:
# 2. Chuyển rating thành nhị phân (tương tác hoặc không)
ratings["interaction"] = (ratings["rating"] >= 3).astype(int)


In [5]:
# 3. Chia dữ liệu thành tập train/test
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)


In [6]:
# 4. Lấy số lượng users và movies duy nhất
num_users = ratings["userId"].nunique()
num_movies = ratings["movieId"].nunique()


In [7]:
# 5. Xây dựng mô hình NCF
embedding_dim = 16

In [8]:
# Input layers
user_input = Input(shape=(1,), name="user_input")
movie_input = Input(shape=(1,), name="movie_input")


In [9]:
# Embedding layers
user_embedding = Embedding(input_dim=num_users + 1, output_dim=embedding_dim, name="user_embedding")(user_input)
movie_embedding = Embedding(input_dim=num_movies + 1, output_dim=embedding_dim, name="movie_embedding")(movie_input)


In [10]:
# Flatten embeddings
user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)


In [11]:
# GMF Layer
gmf_output = tf.keras.layers.Multiply()([user_vec, movie_vec])


In [12]:
# MLP Layer
mlp_input = Concatenate()([user_vec, movie_vec])
mlp_hidden = Dense(32, activation="relu")(mlp_input)
mlp_hidden = Dense(16, activation="relu")(mlp_hidden)
mlp_output = Dense(8, activation="relu")(mlp_hidden)


In [13]:
# Kết hợp GMF và MLP
concat_layer = Concatenate()([gmf_output, mlp_output])
output = Dense(1, activation="sigmoid", name="output_layer")(concat_layer)


In [14]:
# Tạo model
model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [15]:
# 6. Chuẩn bị dữ liệu đầu vào
train_user = train_data["userId"].values
train_movie = train_data["movieId"].values
train_label = train_data["interaction"].values

test_user = test_data["userId"].values
test_movie = test_data["movieId"].values
test_label = test_data["interaction"].values


In [16]:
# 7. Huấn luyện mô hình
model.fit([train_user, train_movie], train_label, epochs=10, batch_size=64, validation_data=([test_user, test_movie], test_label))


Epoch 1/10


/Users/chi.nguyenth/miniconda3/envs/myenv/lib/python3.11/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'movie_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1261/1261 ━━━━━━━━━━━━━━━━━━━━ 2s 992us/step - accuracy: 0.8151 - loss: 0.4628 - val_accuracy: 0.8295 - val_loss: 0.3958
Epoch 2/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8500 - loss: 0.3491 - val_accuracy: 0.8306 - val_loss: 0.3950
Epoch 3/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 1s 989us/step - accuracy: 0.8661 - loss: 0.3192 - val_accuracy: 0.8269 - val_loss: 0.4075
Epoch 4/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8771 - loss: 0.2958 - val_accuracy: 0.8247 - val_loss: 0.4273
Epoch 5/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8803 - loss: 0.2835 - val_accuracy: 0.8246 - val_loss: 0.4359
Epoch 6/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - accuracy: 0.8906 - loss: 0.2636 - val_accuracy: 0.8224 - val_loss: 0.4533
Epoch 7/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 1s 998us/step - accuracy: 0.8985 - loss: 0.2482 - val_accuracy: 0.8150 - val_loss: 0.4804
Epoch 8/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - accuracy: 0.9045 - loss: 0.2342 - v

In [17]:
# 8. Kiểm thử mô hình
loss, accuracy = model.evaluate([test_user, test_movie], test_label)
print(f"Test Accuracy: {accuracy:.4f}")


631/631 ━━━━━━━━━━━━━━━━━━━━ 0s 279us/step - accuracy: 0.8039 - loss: 0.5913
Test Accuracy: 0.8081


In [18]:
from sklearn.metrics import roc_auc_score  
pred = model.predict([test_user, test_movie])  
print("Test AUC-ROC:", roc_auc_score(test_label, pred))  


631/631 ━━━━━━━━━━━━━━━━━━━━ 0s 378us/step
Test AUC-ROC: 0.7486827549212645


In [19]:
# 5. Lưu mô hình
#model.save("model/movie_recommendation_model.keras")
#print("✅ Mô hình đã được lưu dưới định dạng .keras!")

In [20]:
# 10. Hàm đề xuất phim
def recommend_movies(user_id, top_n=5):
    if user_id not in user_mapping:
        print("User ID không hợp lệ.")
        return []
    user_idx = user_mapping[user_id]
    all_movie_ids = np.array(list(movie_mapping.values()))
    user_input = np.full_like(all_movie_ids, user_idx)
    predictions = model.predict([user_input, all_movie_ids]).flatten()
    top_movie_indices = predictions.argsort()[-top_n:][::-1]
    recommended_movie_ids = [list(movie_mapping.keys())[i] for i in top_movie_indices]
    recommended_movies = movies[movies["movieId"].isin(recommended_movie_ids)][["movieId", "title"]]
    return recommended_movies

# 11. Thử nghiệm đề xuất phim
user_id_example = 1
print(recommend_movies(user_id_example))


304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step
      movieId                         title
1252     1663                Stripes (1981)
1258     1673          Boogie Nights (1997)
2156     2871            Deliverance (1972)
2218     2947             Goldfinger (1964)
2219     2948  From Russia with Love (1963)


Xác định user trong tập dữ liệu

Hàm kiểm tra xem user_id có trong dataset không.
Nếu hợp lệ, nó ánh xạ user_id sang chỉ số (user_idx) dùng trong model.
Tạo input để dự đoán

Lấy toàn bộ movieId có trong dataset.
Tạo một mảng chứa user_idx, có cùng kích thước với danh sách movieId, để model có thể dự đoán tất cả các phim cùng lúc.
Dự đoán xác suất tương tác

Dùng model đã huấn luyện (model.predict()) để tính toán mức độ tương tác của user_id với từng bộ phim.
Sắp xếp kết quả dự đoán và chọn top_n phim có xác suất cao nhất.
Trả về danh sách phim đề xuất

Dùng movieId của các phim được đề xuất để lấy tên phim từ tập dữ liệu movies.csv.